In [3]:
import yfinance as yf
import pandas as pd

In [4]:
yf.download("SPY")

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-01-29,43.968750,43.968750,43.750000,43.937500,26.299288,1003200
1993-02-01,43.968750,44.250000,43.968750,44.250000,26.486324,480500
1993-02-02,44.218750,44.375000,44.125000,44.343750,26.542448,201300
1993-02-03,44.406250,44.843750,44.375000,44.812500,26.822998,529400
1993-02-04,44.968750,45.093750,44.468750,45.000000,26.935240,531500
...,...,...,...,...,...,...
2020-06-08,320.220001,323.410004,319.630005,323.200012,323.200012,73641200
2020-06-09,320.299988,323.279999,319.359985,320.790009,320.790009,77479200
2020-06-10,321.420013,322.390015,318.220001,319.000000,319.000000,95000800
